In [7]:
from sklearn.datasets import load_iris
from sklearn import svm

In [8]:
import pandas as pd

In [9]:
iris=load_iris()
df=pd.DataFrame(iris.data,columns=iris.feature_names)
df['flower']=iris.target
df['flower']= df['flower'].apply(lambda x: iris.target_names[x])
df[47:52]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
47,4.6,3.2,1.4,0.2,setosa
48,5.3,3.7,1.5,0.2,setosa
49,5.0,3.3,1.4,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor


## Method 1 : Use train_test_split and manually tune parameters by trial and error

In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(iris.data,iris.target,test_size=0.3)

In [11]:
model=svm.SVC(kernel='rbf',C=30,gamma='auto')
model.fit(X_train,y_train)
model.score(X_test,y_test)

1.0

## Method 2 : Use K fold Cross Validation

### Manually giving different parameters values to cross_val_score function with 5 split

In [13]:
from sklearn.model_selection import cross_val_score
cross_val_score(
    svm.SVC(kernel='rbf',C=10,gamma='auto'),iris.data,iris.target,cv=5
)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [15]:
cross_val_score(
    svm.SVC(kernel='linear',gamma='auto',C=10),iris.data,iris.target,cv=5
)

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [16]:
cross_val_score(
    svm.SVC(kernel='rbf',gamma='auto',C=20),iris.data,iris.target,cv=5
)

array([0.96666667, 1.        , 0.9       , 0.96666667, 1.        ])

### Let's do the above method using loops

In [20]:
import numpy as np
kernels=['rbf','linear']
C=[1,10,20]
avg_scores={}
for kval in kernels:
    for cval in C:
        cv_scores=cross_val_score(svm.SVC(kernel=kval,gamma='auto',C=cval),iris.data,iris.target)
        avg_scores[kval+' with C='+str(cval)]= np.average(cv_scores)
avg_scores

{'rbf with C=1': 0.9800000000000001,
 'rbf with C=10': 0.9800000000000001,
 'rbf with C=20': 0.9666666666666668,
 'linear with C=1': 0.9800000000000001,
 'linear with C=10': 0.9733333333333334,
 'linear with C=20': 0.9666666666666666}

## Method 3: Using GridSearchCV

### GridSearchCV does exactly same thing as for loop above but in a single line of code

In [22]:
from sklearn.model_selection import GridSearchCV
clf= GridSearchCV(
    svm.SVC(gamma='auto'), 
    {'C':[1,10,20],
    'kernel': ['rbf','linear']},
    cv=5, return_train_score=False
)
clf.fit(iris.data,iris.target)
clf.cv_results_


{'mean_fit_time': array([0.00101247, 0.00056744, 0.00185394, 0.00169425, 0.00107245,
        0.        ]),
 'std_fit_time': array([0.00031922, 0.00080445, 0.0023232 , 0.00058559, 0.00066399,
        0.        ]),
 'mean_score_time': array([0.0010962 , 0.00129027, 0.0011209 , 0.00081992, 0.00081911,
        0.0030375 ]),
 'std_score_time': array([0.00046376, 0.00218441, 0.00082866, 0.00041074, 0.00070134,
        0.006075  ]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value=999999),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20, 'kernel': 'linear'}],
 'split0_test_score': ar

In [23]:
result=pd.DataFrame(clf.cv_results_)
result

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001012,0.000319,0.001096,0.000464,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.000567,0.000804,0.001290,0.002184,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.001854,0.002323,0.001121,0.000829,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.001694,0.000586,0.000820,0.000411,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.001072,0.000664,0.000819,0.000701,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5
5,0.000000,0.000000,0.003038,0.006075,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,6


In [25]:
result[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [27]:
dir(clf)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__sklearn_clone__',
 '__sklearn_tags__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_build_request_for_signature',
 '_check_feature_names',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_doc_link_module',
 '_doc_link_template',
 '_doc_link_url_param_generator',
 '_estimator_type',
 '_format_results',
 '_get_default_requests',
 '_get_doc_link',
 '_get_metadata_request',
 '_get_param_names',
 '_get_routed_params_for_fit',
 '_get_scorers',
 '_get_tags',
 '_more_tags',
 '_parameter_constraints',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_run_sea

In [29]:
clf.best_params_

{'C': 1, 'kernel': 'rbf'}

In [31]:
clf.best_score_

0.9800000000000001

## Now here we have specified only 3 values of C and 2 values for kernel and it have PnC on them to get us the results for all but if there will be more than 100 parameters , it will increase computational cost

### So, in order to do that, RandomizedSearchCV reduces the number of iterations and use random combination of parameters. It reduces the cost of computation

In [35]:
from sklearn.model_selection import RandomizedSearchCV

In [36]:
rs= RandomizedSearchCV(
    svm.SVC(gamma='auto'), 
    {'C':[1,10,20],
    'kernel': ['rbf','linear']},
    cv=5, return_train_score=False,
    n_iter=2
)
rs.fit(iris.data,iris.target)

RandomizedSearchCV(cv=5, estimator=SVC(gamma='auto'), n_iter=2,
                   param_distributions={'C': [1, 10, 20],
                                        'kernel': ['rbf', 'linear']})

In [38]:
pd.DataFrame(rs.cv_results_)[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,20,linear,0.966667
1,20,rbf,0.966667


# Let's now check for different model with different paramaters

In [40]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [48]:
model_params= {
    'svm': {
        'model' : svm.SVC(gamma='auto'),
        'params': {
            'C':[1,10,20],
            'kernel':['rbf','linear']
        }
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear'),
        'params': {
            'C': [1,5,10]
        }
    }
}

In [51]:
scores=[]

for model_name,mp in model_params.items():
    clf=GridSearchCV(mp['model'],mp['params'],cv=5,return_train_score=False)
    clf.fit(iris.data,iris.target)
    scores.append(
        {
            'model':model_name,
            'best_score':clf.best_score_,
            'best_param':clf.best_params_
        }
    )

final = pd.DataFrame(scores,columns=['model','best_score','best_params'])
final

,model,best_score,best_params
0,svm,0.980000,NaN
1,random_forest,0.960000,NaN
2,logistic_regression,0.966667,NaN
